In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from scipy.stats import zscore

# until this repository is distributed as a package:
sys.path.append("/path/to/src/brainwide-npix/")

from bwnpix import load
from bwnpix import anatomy
from bwnpix.multiprobe import *

In [ ]:
datasets = [
    ["mousename1", "20220601", "/path/to/recording/data/m94/catgt_mousename1_20220601_g0"],   
]

In [ ]:
%%time
use_cache = False # Set to true after this has been run once

all_data = []
paths = []
tract_names = ["OFC", "Thal", "MO", "npix_tract1"] # tract name for easy reference
for dataset in datasets:
    pathname = f'/path/to/data/cache/export/{dataset[:2]}.pkl'
    paths.append(pathname)
    if not use_cache:
        # if using bombcell for unit quality, set qc='bombcell', otherwise 'ks'
        mpd = load.MultiprobeData(dataset[0],dataset[1], qc='bombcell')
        for j in range(4): # add 4 probes
            mpd.add_experiment(os.path.join(dataset[2], f"{dataset[0]}_{dataset[1]}_g0_imec{j}",f"imec{j}_ks2"), tract_names[j])
        mpd.combine_experiments()
        event_map = {"XA_1": "session", "XA_2": "trial", "XA_3": "stim"} # map of names of events
        mpd.add_events(dataset[2], event_map)
        mpd.save_data(pathname)

In [ ]:
%%time # Takes about 30-60s
# Load AIBS atlas to use for histology
atlas = anatomy.AIBSAtlas()

In [ ]:
%%time

data = MultiprobeEphysExperiment() # or subclass name for loading behavior / other events
# load eye closure data
data.load_units_from_ks(paths[0], load_lfp=False) # Set load_lfp to True for lfp data in addition to spike data
data._mouse_name = "mousename1"; data._recording = "mousename1_20220601"
data.load_histology(atlas)

In [ ]:
%%time
bin_t = 0.01
fr, bins = data.firing_rates_over_recording(bin_size=bin_t, smooth_size=10)
zfr = zscore(fr,1)

In [ ]:
# Get brain region plotting variables

areas_cbar, areas_cm = data.get_area_colorbar(data.get_brain_areas(info='id'))
area_names = data.get_brain_areas(info='acronym', level='bottom')
# plot brain regions with plt.imshow(areas_cbar, aspect='auto', cmap=areas_cm)

# equivalent functionality for LFP data

#lfp_areas = data.get_brain_areas(mask=False, info='id', locs=data.get_locs_for_lfp())
#lfp_cbar, lfp_cm = data.get_area_colorbar(lfp_areas)